In [2]:
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance

import folium
import geopandas as gpd
from shapely.geometry import Point
import random

In [16]:
AED_check = pd.read_csv("C:/Users/leonw/Downloads/aed(Sheet1)(6).csv",encoding='unicode_escape')

In [17]:
AED_check.head()

,id,type,address,number,postal_code,municipality,province,location,public,available,Opens,Closes,Notes,complete_address,latitude,longitude,Checked,Unnamed: 17
0,2741.0,NaN,Aarschotsesteenweg,58,3012.0,Leuven,Vlaams-Brabant,"glvl, klantenkantoor",Y,NaN,0,0,NaN,"58 Aarschotsesteenweg, Leuven, Vlaams-Brabant,...",50.893353,4.707561,FALSE,NaN
1,1905.0,NaN,Aarschotsesteenweg,730,3012.0,Wilsele,Vlaams-Brabant,NaN,Y,couldn't find,0,0,NaN,"730 Aarschotsesteenweg, Wilsele, Vlaams-Braban...",50.928122,4.724020,Yes,NaN
2,3076.0,Cardiac Science,Aarschotsesteenweg,163,3012.0,Leuven,Vlaams-Brabant,"1°verd,gang kleedkamers",N,NaN,8:30,22:30,Olivier file: Public - Y. Ma tot zo - 8u3 tot...,"163 Aarschotsesteenweg, Leuven, Vlaams-Brabant...",50.910898,4.713118,Yes,NaN
3,10696.0,NaN,Abdij Van Vlierbeek,15,3010.0,Kessel-Lo,Vlaams-Brabant,NaN,Y,Yes,0,24,NaN,"15 Abdij Van Vlierbeek, Kessel-Lo, Vlaams-Brab...",50.891918,4.734149,Yes,NaN
4,4991.0,NaN,Albert Dejonghestraat,2,3010.0,Kessel-Lo,Vlaams-Brabant,clubhouse,N,NaN,0,0,NaN,"2 Albert Dejonghestraat, Kessel-Lo, Vlaams-Bra...",50.876465,4.727593,No,NaN


In [18]:
AED_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                149 non-null    float64
 1   type              68 non-null     object 
 2   address           164 non-null    object 
 3   number            159 non-null    object 
 4   postal_code       163 non-null    float64
 5   municipality      162 non-null    object 
 6   province          142 non-null    object 
 7   location          104 non-null    object 
 8   public            163 non-null    object 
 9   available         131 non-null    object 
 10  Opens             164 non-null    object 
 11  Closes            164 non-null    object 
 12  Notes             67 non-null     object 
 13  complete_address  126 non-null    object 
 14  latitude          159 non-null    float64
 15  longitude         159 non-null    float64
 16  Checked           164 non-null    object 
 1

In [19]:
AED_check = AED_check[["latitude","longitude","available","Checked","address","number"]].copy()

In [20]:
AED_check = AED_check.dropna(subset = ['latitude', 'longitude'])

In [21]:
AED_check.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 0 to 163
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   159 non-null    float64
 1   longitude  159 non-null    float64
 2   available  129 non-null    object 
 3   Checked    159 non-null    object 
 4   address    159 non-null    object 
 5   number     156 non-null    object 
dtypes: float64(2), object(4)
memory usage: 8.7+ KB


In [22]:
AED_checked = AED_check[AED_check["Checked"]=="Yes"].copy()

In [23]:
len(AED_checked)

136

In [24]:
AED_to_check = AED_check[AED_check["Checked"]=="No"].copy()

In [25]:
AED_check_again = AED_check[AED_check["available"]=="couldn't find"].copy()

In [26]:
AED_to_check.head()

,latitude,longitude,available,Checked,address,number
4,50.876465,4.727593,NaN,No,Albert Dejonghestraat,2
15,50.880951,4.712171,NaN,No,Bondgenotenlaan,145
16,50.855307,4.728376,NaN,No,Brabantlaan,1
28,50.884142,4.714630,NaN,No,Diestsepoort,6
29,50.884142,4.714630,NaN,No,Diestsepoort,6


In [27]:
Check = folium.Map(location=[AED_check['latitude'].mean(), AED_check['longitude'].mean()], zoom_start=13)
# Add AEDs markers (red)


for _, row in AED_to_check.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['address']} {row['number']}",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(Check)
for _, row in AED_checked.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['address']} {row['number']}",
        icon=folium.Icon(color='red', icon='heartbeat', prefix='fa')
    ).add_to(Check)
for _, row in AED_check_again.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['address']} {row['number']}",
        icon=folium.Icon(color='orange', icon='heartbeat', prefix='fa')
    ).add_to(Check)

Check

In [28]:
# Save and display the map
Check.save("C:/Users/leonw/Downloads/aed_check.html")

In [99]:
aed_loc = pd.read_parquet("OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/aed_locations.parquet.gzip")

In [107]:
AED_loc = aed_loc[aed_loc['postal_code']>2999]

In [108]:
AED_loc = AED_loc[AED_loc['postal_code']<3100]

In [134]:
AED_Iso = gpd.read_file('C:/Users/leonw/OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/temp.gpkg', layer='AED_data')

In [135]:
AED_Iso.head()

,latitude,longitude,available,Checked,Opens,Closes,geometry
0,50.893353,4.707561,None,FALSE,None,None,"POLYGON ((4.70674 50.89899, 4.70673 50.89892, ..."
1,50.928122,4.724020,couldn't find,Yes,None,None,"POLYGON ((4.71509 50.92497, 4.71528 50.92466, ..."
2,50.910898,4.713118,None,Yes,8:30,22:30,"POLYGON ((4.70643 50.91046, 4.70659 50.90975, ..."
3,50.891918,4.734149,Yes,Yes,0,24,"POLYGON ((4.72414 50.88829, 4.72428 50.88796, ..."
4,50.876465,4.727593,None,No,None,None,"POLYGON ((4.71895 50.87788, 4.71928 50.87302, ..."


In [136]:
AEDs = AED_Iso
AEDs['AED'] = list(zip(AEDs["longitude"], AEDs["latitude"]))
new_open = AEDs["Opens"].str.split(":", n=1, expand=True)
new_open = new_open.apply(pd.to_numeric, errors='coerce') 
new_open['time']=new_open[0]+(new_open[1]/60)
AEDs['Opens'] = new_open['time']

new_close = AEDs["Closes"].str.split(":", n=1, expand=True)
new_close = new_close.apply(pd.to_numeric, errors='coerce') 
new_close['time']=new_close[0]+(new_close[1]/60)
AEDs['Closes'] = new_close['time']

AEDs.head()

,latitude,longitude,available,Checked,Opens,Closes,geometry,AED
0,50.893353,4.707561,None,FALSE,NaN,NaN,"POLYGON ((4.70674 50.89899, 4.70673 50.89892, ...","(4.7075612, 50.8933532)"
1,50.928122,4.724020,couldn't find,Yes,NaN,NaN,"POLYGON ((4.71509 50.92497, 4.71528 50.92466, ...","(4.7240203, 50.9281215)"
2,50.910898,4.713118,None,Yes,8.5,22.5,"POLYGON ((4.70643 50.91046, 4.70659 50.90975, ...","(4.713117604, 50.91089775)"
3,50.891918,4.734149,Yes,Yes,NaN,NaN,"POLYGON ((4.72414 50.88829, 4.72428 50.88796, ...","(4.7341489, 50.8919184)"
4,50.876465,4.727593,None,No,NaN,NaN,"POLYGON ((4.71895 50.87788, 4.71928 50.87302, ...","(4.7275933, 50.8764651)"


In [ ]:
new = AED_Iso["Closes"].str.split(":", n=1, expand=True)
AEne.head()

,0,1
0,None,None
1,None,None
2,22,30
3,24,None
4,None,None


In [124]:
new = new.apply(pd.to_numeric, errors='coerce')

In [125]:
new['time']=new[0]+(new[1]/60)

In [ ]:
new

,0,1,time
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,8.0,30.0,8.5
3,0.0,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
176,NaN,NaN,NaN
177,0.0,NaN,NaN
178,NaN,NaN,NaN
179,NaN,NaN,NaN


In [153]:
decline_rate = 0.2
acceptance_list = []
for i in range(30):
            acceptance = int(np.random.choice([0,1], p=[decline_rate, (1-decline_rate)], size = 1))
            acceptance_list.append(acceptance)

print(acceptance_list)
print(sum(acceptance_list)/30)
df_acceptance = pd.DataFrame({'Probability' : acceptance_list})
print(df_acceptance.head())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
0.8666666666666667
   Probability
0            1
1            1
2            1
3            1
4            1


C:\Users\leonw\AppData\Local\Temp\ipykernel_39268\2736521632.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  acceptance = int(np.random.choice([0,1], p=[decline_rate, (1-decline_rate)], size = 1))
